In [7]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd ../AILS-MICCAI-UWF4DR-Challenge

In [31]:
# setup
#!apt-get update
#!pip install python-dotenv
#!pip install loguru
#!pip install gdown
#!pip install typer

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 198.1 kB/s eta 0:00:00 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [29]:
# load data and unzip data

!python tools/download_data_and_chkpts.py

Downloading...
From (original): https://drive.google.com/uc?id=1jm48RSCctyxtEkppS45Znh0wtdf9patA
From (redirected): https://drive.google.com/uc?id=1jm48RSCctyxtEkppS45Znh0wtdf9patA&confirm=t&uuid=071ccac2-f501-40a2-a483-c2e246db8916
To: /home/optima/mhaderer/AILS-MICCAI-UWF4DR-Challenge/data/downloads/DeepDRiD.zip.enc
100%|████████████████████████████████████████| 303M/303M [00:03<00:00, 93.0MB/s]
Downloaded to 'data/downloads/DeepDRiD.zip.enc'
Decrypted from 'data/downloads/DeepDRiD.zip.enc' to 'data/downloads/DeepDRiD.zip'
Extracting 'data/downloads/DeepDRiD.zip' to 'data/external'
Downloading...
From (original): https://drive.google.com/uc?id=1K8xwscXQQo0KXEzFaC2wybgD-UYNXvfc
From (redirected): https://drive.google.com/uc?id=1K8xwscXQQo0KXEzFaC2wybgD-UYNXvfc&confirm=t&uuid=1c1b6c43-5e02-4329-8b20-2a297098d5ec
To: /home/optima/mhaderer/AILS-MICCAI-UWF4DR-Challenge/data/downloads/UWF4DRChallengeData.zip.enc
100%|████████████████████████████████████████| 150M/150M [00:02<00:00, 70.9MB/

In [39]:
# imports
from ails_miccai_uwf4dr_challenge.dataset import DatasetBuilder, DatasetOriginationType, ChallengeTaskType, CustomDataset

import torch.nn as nn
from sklearn.metrics import roc_auc_score, average_precision_score
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

import torch
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from ails_miccai_uwf4dr_challenge.preprocess_augmentations import ResidualGaussBlur, MultiplyMask
from ails_miccai_uwf4dr_challenge.augmentations import rotate_affine_flip_choice, resize_only
from ails_miccai_uwf4dr_challenge.models.metrics import sensitivity_score, specificity_score
from ails_miccai_uwf4dr_challenge.models.trainer import Metric, DefaultMetricsEvaluationStrategy, Trainer, TrainingContext, MetricCalculatedHook

In [33]:
# select device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))

Device: cpu


/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1711403388920/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [35]:



preprocessing = A.Compose([
        A.Resize(800, 1016, p=1),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1),
        #ResidualGaussBlur(p=1),
        MultiplyMask(p=1),
        A.Resize(800, 1016, p=1),
        #A.Equalize(p=0.1),
        #A.CLAHE(clip_limit=5., p=0.3)
    ])

augment_train = A.Compose([
        #A.VerticalFlip(p=0.5),
        #A.HorizontalFlip(p=0.5),
        #A.Affine(rotate=15, rotate_method='ellipse', p=0.5),
        ToTensorV2(p=1)
    ])

augment_val = A.Compose([
        ToTensorV2(p=1)
    ])


transforms_train = A.Compose([
    preprocessing,
    augment_train
])

transforms_val = A.Compose([
    preprocessing,
    augment_val
])

In [ ]:
from ails_miccai_uwf4dr_challenge.models.architectures.task1_automorph_plain import AutoMorphModel

model = AutoMorphModel(enc_frozen=True)

model.to(device)
print("Training model: ", model.__class__.__name__)

metrics = [
        Metric('auroc', roc_auc_score),
        Metric('auprc', average_precision_score),
        Metric('accuracy', lambda y_true, y_pred: (y_pred.round() == y_true).mean()),
        Metric('sensitivity', sensitivity_score),
        Metric('specificity', specificity_score)
    ]

class WandbLoggingHook(MetricCalculatedHook):
        def on_metric_calculated(self, training_context: TrainingContext, metric: Metric, result, last_metric_for_epoch: bool):
            import wandb
            wandb.log(data={metric.name: result}, commit=last_metric_for_epoch)

metrics_eval_strategy = DefaultMetricsEvaluationStrategy(metrics)
if(use_wandb):
    metrics_eval_strategy.register_metric_calculated_hook(WandbLoggingHook())

In [43]:


config = {
    "learning_rate": 1e-3,
    "dataset": "Original",
    "epochs": 10,
    "batch_size": 16,
    "model_type": model.__class__.__name__
}

def combined_losses(pred, target):
    bce = F.binary_cross_entropy_with_logits(pred, target) * 0.5
    smooth_l1 = F.smooth.l1_loss(pred, target) * 0.5
    return bce + smooth_l1

criterion = combined_losses

optimizer = optim.AdamW(model.parameters(), lr=config["learning_rate"])
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)


dataset_builder = DatasetBuilder(DatasetOriginationType.ORIGINAL, ChallengeTaskType.TASK2)

train_data, val_data = dataset_builder.get_train_val()
train_dataset = CustomDataset(train_data, transform=transforms_train)
val_dataset = CustomDataset(val_data, transform=transforms_val)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)


trainer = Trainer(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, device,
                        metrics_eval_strategy=metrics_eval_strategy)

if use_wandb:
    pass
    #wandb.init(entity='miccai-challenge-2024' ,project='task2', config=config)

print(f"Start training [{config['model_type']}] on [{config['dataset']}] dataset for [{config['epochs']}] epochs with batch size [{config['batch_size']}]")

#trainer.train(num_epochs=config["epochs"])

print("Training finished.")


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Dataset length:  319
Dataset length:  80
Start training [AutoMorphModel] on [Original] dataset for [10] epochs with batch size [16]
Training finished.
